In [8]:
%load_ext autoreload
%autoreload 2

import sys, gzip, pickle, torch, re, time 
import matplotlib.pyplot as plt
from glob import glob
from tqdm.notebook import tqdm
sys.path.insert(0,'/home/sulcan/Documents/ipac-logbook/code/')
sys.path.insert(0,'/home/sulcan/Documents/lora_pacuna2/code/')
sys.path.insert(0,'/home/sulcan/Documents/chatqcd/code/')
from llm_chatqcd import *
from mmd import prepare_mmd_eqations_for_pacuna, find_abstract, split_markdown_sections
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from openTSNE import TSNE
from sentence_transformers import SentenceTransformer
from pprint import pprint
from collections import defaultdict
from glob import glob
from pylatexenc.latex2text import LatexNodes2Text

gd = lambda x,i : x[list(x.keys())[i]]

gpu_id = -1
data_folder = '/home/sulcan/Documents/chatqcd/data/'
device = f'cuda:0'
batch_size = 5000

arxiv_prefix = 'arxiv'
# books_prefix = 'books'

folders = [f'{data_folder}/{arxiv_prefix}/']
           # f'{data_folder}/{books_prefix}/']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Loading LLM

In [2]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3" # 'microsoft/phi-1.5'
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = model_id, 
                                             device_map = device,
                                             quantization_config = quantization_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

#### Loading data 

In [3]:
if False:
    files = []
    # list of files
    for folder in tqdm(folders):
        files.extend(glob(folder + '*.mmd'))
    # loading the list
    data_mmd = {}
    for file in tqdm(files):
        with open(file, 'r') as f:
            data_mmd[file] = f.read()
    # preprocessing
    # print('replacing mmd equations with latex ones ...')
    # data_mmd = prepare_mmd_eqations_for_pacuna(data_mmd)
    # print('done ...')
    # saving
    with gzip.open(f'{data_folder}/arxiv_raw_mmd.pickle.gzip','wb') as f:
        pickle.dump(data_mmd, f)
else:
    with gzip.open(f'{data_folder}/arxiv_raw_mmd.pickle.gzip','rb') as f:
        data_mmd = pickle.load(f) 

In [4]:
with open(f'{data_folder}/arxiv_summaries.pickle','rb') as f:
    summaries = pickle.load(f)

#### Getting the abstracts

In [5]:
if False:
    sections = {}
    for k in tqdm(data_mmd):
        paper = data_mmd[k]
        # replacing bold (*) with header
        paper = re.sub(r'\n\*\*(.*)\*\*\n',r'\n# \1\n',paper)
        # replacing italics (_something_) with header
        paper = re.sub(r'\n_(.+)_ ',r'\n# \1\n',paper)
        # removing \\tag{\d+} because Nougat gives it as a reference
        paper_no_eq_tag = re.sub(r'\\tag{\d+}','',paper)


        sections_ = split_markdown_sections(paper)
        abstract = get_abstract(sections_)

        sections[k] = {
            'raw' : data_mmd[k],
            'preprocessed' : paper,
            'preprocessed_no_eq_tag' : paper_no_eq_tag,
            'sections' : sections_,
            'abstract' : abstract,
            'summary' : summaries[k].split('[/INST]')[-1]}
        with gzip.open(f'{data_folder}/arxiv_with_prompts.pickle.gzip','wb') as f:
            pickle.dump(sections,f)
else:
    with gzip.open(f'{data_folder}/arxiv_with_prompts.pickle.gzip','rb') as f:
            sections = pickle.load(f)

# Q&A

In [6]:
if False:
    sections = generate_prompts(sections, tokenizer)

    with gzip.open(f'{data_folder}/arxiv_with_prompts.pickle.gzip','wb') as f:
        pickle.dump(sections,f)
else:
    with gzip.open(f'{data_folder}/arxiv_with_prompts.pickle.gzip','rb') as f:
        sections = pickle.load(f)

In [7]:
qas = defaultdict(lambda : [])
qas_files = glob(f'{data_folder}/qas_*.pickle')
print(qas_files)
for file in tqdm(qas_files):
    with open(file,'rb') as f:
        try:
            qas_ = pickle.load(f)
            qas.update({k : qas_[k] for k in qas_ if len(qas_[k]) > 0})
        except Exception as e:
            print(f'{file} corrputed {str(e)}')
start = batch_size * gpu_id
stop = batch_size * (gpu_id + 1) 
# sections = {k : sections[k] for k in list(sections.keys())[start:stop]}
# sections = {k : sections[k] for k in list(sections.keys())[start:stop]}


['/home/sulcan/Documents/chatqcd/data/qas_1.pickle', '/home/sulcan/Documents/chatqcd/data/qas_6.pickle', '/home/sulcan/Documents/chatqcd/data/qas_5.pickle', '/home/sulcan/Documents/chatqcd/data/qas_0.pickle', '/home/sulcan/Documents/chatqcd/data/qas_2.pickle', '/home/sulcan/Documents/chatqcd/data/qas_3.pickle', '/home/sulcan/Documents/chatqcd/data/qas_7.pickle', '/home/sulcan/Documents/chatqcd/data/qas_4.pickle']


100%|█████████████████████████████████████████████| 8/8 [00:06<00:00,  1.17it/s]


In [ ]:
for k in tqdm(sections):
    prompts = sections[k]['prompts']
    
    if k in qas:
        if len(qas[k]) == len(prompts):
            continue
            
    print(k)
    for prompt in tqdm(prompts):
        messages = [
            {"role": "user", "content": prompt}
        ]
    
        tic = time.time()
        encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
        model_inputs = encodeds.to(device)
        generated_ids = model.generate(model_inputs, max_new_tokens=1024, do_sample=True)
        decoded = tokenizer.batch_decode(generated_ids)

        qas[k].append({
            'prompt' : prompt,
            'output' : decoded
        })
    
    with open(f'{data_folder}/qas_{gpu_id}.pickle','wb') as f:
        pickle.dump(dict(qas),f)
    

  0%|                                                 | 0/45422 [00:00<?, ?it/s]

/home/sulcan/Documents/chatqcd/data//arxiv/2012.15670v2.New_method_for_renormalon_subtraction_using_Fourier_transform.mmd



0it [00:00, ?it/s]
  0%|                                     | 36/45422 [01:01<21:40:44,  1.72s/it]

/home/sulcan/Documents/chatqcd/data//arxiv/0901.2260v1.The_heavy_quark_antiquark_potential_from_lattice_and_perturbative_QCD.mmd



0it [00:00, ?it/s]
  0%|                                     | 50/45422 [02:04<34:02:00,  2.70s/it]

...


  0%|                                                     | 0/4 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

 25%|███████████▎                                 | 1/4 [00:52<02:38, 52.72s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

 50%|██████████████████████▌                      | 2/4 [01:40<01:39, 49.61s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

 75%|██████████

/home/sulcan/Documents/chatqcd/data//arxiv/hep-ph_9409446v2.High_energy_scattering_in_QCD_and_cross_singularities_of_Wilson_loops.mmd



  0%|                                                    | 0/30 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  3%|█▍                                          | 1/30 [00:59<28:50, 59.68s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  7%|██▉                                         | 2/30 [02:02<28:36, 61.29s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

 10%|████▍     

/home/sulcan/Documents/chatqcd/data//arxiv/hep-ph_0703297v1.Introduction_to_Chiral_Perturbation_Theory.mmd



  0%|                                                    | 0/11 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
if True:
    with open(f'{data_folder}/qas_{gpu_id}.pickle','wb') as f:
        pickle.dump(dict(qas),f)

# Summaries

In [ ]:
summaries = {}
# Merging all summaries together into one dictioary
for file in sorted(glob(f'{data_folder}/arxiv_summaries_*.pickle')):
    with open(file,'rb') as f:
        summaries_ = pickle.load(f)
    summaries.update(summaries_)
print(len(summaries))

# chunking the whole process
start = gpu_id * 10000
stop = (gpu_id + 1) * 10000
print(start,stop)
# sections = {k : sections[k] for k in list(sections.keys())[start:stop]}
len(sections)

In [ ]:
for i, k in enumerate(sections): 
    
    preprocessed = sections[k]['preprocessed']
    
    if k in summaries:
        continue
    
    tokens = tokenizer.encode(preprocessed)
    tokens = tokens[:32000]
    preprocessed = tokenizer.decode(tokens,skip_special_tokens=True)
    
    
    messages = [
        {"role": "user", "content": 
         f"""Imagine that you are a physics student. 
         Generate a short summary without making things up for context to train LLM without mentioning paper from the following text:\n\n"{preprocessed}."""}
    ]
    
    tic = time.time()
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to('cuda:0')
    generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    
    summaries[k] = decoded[0]
    
    
    if i % 10 == 0:
        with open(f'{data_folder}/arxiv_summaries.pickle','wb') as f:
            pickle.dump(summaries,f)

    print(f'{i}/{len(sections)} {time.time() - tic}')

with open(f'{data_folder}/arxiv_summaries.pickle','wb') as f:
    pickle.dump(summaries,f)